In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import requests
import json
import pandas as pd
import datetime

In [3]:
current_season = "2023-24"

In [4]:
def get_data(url):
    """ Retrieve the fpl player data from the hard-coded url
    """
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception("Response was code " + str(response.status_code))
    responseStr = response.text
    return responseStr

In [5]:
def get_teams_names():
    result = []
    teams = pd.read_csv('data/'+ current_season + '/teams.csv')
    for t in teams.iterrows():
        if t[1]['name'].replace(" ", "") == "Nott'mForest":
            result.append('Forest')
        elif t[1]['name'].replace(" ", "") == "ManUtd":
            result.append('ManUnited')
        elif t[1]['name'].replace(" ", "") == "Spurs":
            result.append('Tottenham')
        elif t[1]['name'].replace(" ", "") == "SheffieldUtd":
            result.append('SheffieldUnited')
        else:
            result.append(t[1]['name'].replace(" ", ""))
    # manually adding remaining teams
    result.append('Burnley')
    result.append('Norwich')
    result.append('Watford')
    result.append('Luton')
    result.append('Southampton')
    result.append('Leicester')
    result.append('Leeds')
    return result

In [6]:
team_names = get_teams_names()
team_names

['Arsenal',
 'AstonVilla',
 'Bournemouth',
 'Brentford',
 'Brighton',
 'Burnley',
 'Chelsea',
 'CrystalPalace',
 'Everton',
 'Fulham',
 'Liverpool',
 'Luton',
 'ManCity',
 'ManUnited',
 'Newcastle',
 'Forest',
 'SheffieldUnited',
 'Tottenham',
 'WestHam',
 'Wolves',
 'Burnley',
 'Norwich',
 'Watford',
 'Luton',
 'Southampton',
 'Leicester',
 'Leeds']

In [7]:
def reload_all_teams_ratings():
    result = pd.DataFrame()
    for team in get_teams_names():
        response = get_data('http://api.clubelo.com/' + team)
        print(team, len(response))
        rows = response.splitlines()
        columns = rows[0].split(',')
        content = []
        for r in rows[1:]:
            content.append(r.split(','))
        df = pd.DataFrame(content, columns = columns)
        result = pd.concat([result, df])
    result = result.dropna()
    result.to_csv('data/misc/team_ratings_history.csv')

In [8]:
def load_all_teams_ratings():
    df = pd.read_csv('data/misc/team_ratings_history.csv')
    return df

In [9]:
reload_all_teams_ratings()

Arsenal 330122
AstonVilla 383801
Bournemouth 78274
Brentford 74793
Brighton 146272
Burnley 253731
Chelsea 324551
CrystalPalace 280631
Everton 348637
Fulham 259129
Liverpool 312203
Luton 170677
ManCity 329273
ManUnited 330024
Newcastle 369260
Forest 294628
SheffieldUnited 311112
Tottenham 362799
WestHam 388318
Wolves 297421
Burnley 253731
Norwich 280610
Watford 205252
Luton 170677
Southampton 346863
Leicester 341992
Leeds 282273


In [10]:
ratings = load_all_teams_ratings()
ratings['Club'].unique()

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
       'Liverpool', 'Luton', 'Man City', 'Man United', 'Newcastle',
       'Forest', 'Sheffield United', 'Tottenham', 'West Ham', 'Wolves',
       'Norwich', 'Watford', 'Southampton', 'Leicester', 'Leeds'],
      dtype=object)

In [11]:
example_date = datetime.datetime(2022, 9, 13)

In [12]:
example_date.strftime("%Y-%m-%d")

'2022-09-13'

In [13]:
def get_rating_for_team(team_ratings, team_name, date):
    rating = team_ratings[(team_ratings['From'] <= date.strftime("%Y-%m-%d")) & (team_ratings['Club'] == team_name)].iloc[-1]['Elo']
    return rating

In [14]:
get_rating_for_team(ratings, 'Wolves', example_date)

1709.16552734